# TMDB Movies Data Cleaning


In [13]:
# locating the Spark installation
import findspark
findspark.init()

In [14]:
# Initialize Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MovieDataCleaning").getOrCreate()


In [ ]:
def clean_movie_data(filepath):
    # Read the parquet file into a DataFrame
    df = spark.read.parquet(filepath)

    # Drop irrelevant columns: ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
    df = df.drop(*['adult', 'imdb_id', 'original_title', 'video', 'homepage'])



    return df

In [16]:
df = clean_movie_data("../data/extracted_movies_df.parquet")
df.columns

['id',
 'title',
 'original_language',
 'release_date',
 'runtime',
 'popularity',
 'vote_average',
 'vote_count',
 'overview',
 'genres',
 'production_companies',
 'belongs_to_collection',
 'production_countries',
 'spoken_languages',
 'cast',
 'crew']

+--------------------+
|              genres|
+--------------------+
|[{'id': 12, 'name...|
|[{'id': 28, 'name...|
|[{'id': 12, 'name...|
|[{'id': 12, 'name...|
|[{'id': 28, 'name...|
|[{'id': 28, 'name...|
|[{'id': 28, 'name...|
|[{'id': 14, 'name...|
|[{'id': 12, 'name...|
|[{'id': 10751, 'n...|
|[{'id': 28, 'name...|
|[{'id': 16, 'name...|
|[{'id': 10751, 'n...|
|[{'id': 28, 'name...|
|[{'id': 18, 'name...|
|[{'id': 28, 'name...|
|[{'id': 12, 'name...|
|[{'id': 878, 'nam...|
+--------------------+

